In [ ]:
import torch
import statistics
from statistics import mean
import csv
from sklearn import metrics
from copy import deepcopy
from torch_geometric.nn import MessagePassing
from torch.nn.parameter import Parameter

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
class evalaution_of_xmethods_stability(object):
    def __init__(self, a_trained_model, test_data):
        """
            Perturb Node Features for Entire Dataset
            Explain perturbed dataset
            Compute their distance
        """
        super(evalaution_of_xmethods_stability, self).__init__()

        self.a_trained_model = a_trained_model
        self.test_data = test_data


    def normalize_saliency_node_features_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                node_gradients = []
                for dim in node_grads:
                    node_gradients.append((dim-min(node_grads))/(max(node_grads)-min(node_grads)) if (max(node_grads)-min(node_grads)) != 0 else 0)
                new_gradients.append(node_gradients)
            Graphs_new_gradients.append(new_gradients)

        return Graphs_new_gradients

    def normalize_saliency_node_based(self, sal_maps):
        Graphs_new_gradients = []
        for graph_grads in sal_maps:
            new_gradients = []
            for node_grads in graph_grads:
                val = (node_grads-min(graph_grads))/(max(graph_grads)-min(graph_grads)) if (max(graph_grads)-min(graph_grads)) != 0 else 0
                new_gradients.append(val)
            Graphs_new_gradients.append(new_gradients)

        return Graphs_new_gradients

    def normalize_stability_distance(self, distance_list):
        normalized_distance_list = []
        for exp_diff in distance_list:
            val = (exp_diff-min(distance_list))/(max(distance_list)-min(distance_list)) if (max(distance_list)-min(distance_list)) != 0 else 0
            normalized_distance_list.append(val)
        return normalized_distance_list

    def perturb_node_features_of_dataset(self, dataset, mean, std):
        dataset_perturbed = deepcopy(dataset)
        for graph in dataset_perturbed:
            continuous_noise = torch.ones(dataset[0].x.size()[1]).normal_(mean, std)
            graph.x += continuous_noise
        return dataset_perturbed

    def dist_explanation(self, normal_saliency, perturbed_saliency, top_k_features):
        distance_list = []
        for normal_graph, perturbed_graph in zip(normal_saliency, perturbed_saliency):
            perturbed_graph = np.array(perturbed_graph)
            perturbed_graph = torch.from_numpy(perturbed_graph)
            normal_graph = np.array(normal_graph)
            normal_graph = torch.from_numpy(normal_graph)

            normal_graph_mask = torch.where(normal_graph >= normal_graph.topk(top_k_features)[0][-1].item(),
                                            torch.ones_like(normal_graph), torch.zeros_like(normal_graph))
            perturbed_graph_mask = torch.where(perturbed_graph >= perturbed_graph.topk(top_k_features)[0][-1].item(),
                                                torch.ones_like(perturbed_graph), torch.zeros_like(perturbed_graph))

            distance = F.pairwise_distance(normal_graph_mask.unsqueeze(dim=0), perturbed_graph_mask.unsqueeze(dim=0), p=1) / normal_graph_mask.size()[0]
            distance_list.append(distance.tolist()[0])
        return distance_list

    def one_graph_all_together(self, normal_saliency, perturbed_saliency):
        normal_saliency_list = []
        perturbed_saliency_list = []
        for normal_graph, perturbed_graph in zip(normal_saliency, perturbed_saliency):
            normal_saliency_list.append([node_feat for node in normal_graph for node_feat in node])
            perturbed_saliency_list.append([node_feat for node in perturbed_graph for node_feat in node])
        return perturbed_saliency_list, perturbed_saliency_list

    def distance_thresholding_node_features(self, distance_list, importance_threshold):
        stability_list = []
        for graph_score in distance_list:
            if importance_threshold == graph_score == 0:
                stability_list.append(1)
            if importance_threshold == graph_score == 1:
                stability_list.append(0)
            if importance_threshold < graph_score:
                stability_list.append(1 - graph_score)
            else:
                stability_list.append(0)
        return stability_list

    def normalize_saliency_edge_based(self, saliency_maps):
        Graphs_new_gradients = []

        for graph_grads in saliency_maps:
            #new_gradients = []
            for edge_grads in graph_grads:
                edge_grads = (edge_grads-min(graph_grads))/(max(graph_grads)-min(graph_grads)) if (max(graph_grads)-min(graph_grads)) != 0 else 0
                #new_gradients.append(val)
            Graphs_new_gradients.append(graph_grads)

        return Graphs_new_gradients

    def my_stability(self, normal_saliencies_for_multiple_classes, perturbed_saliencies_for_multiple_classes,
                     top_k_features, importance_threshold, style):
        if style == "Node Feature":
            try:
                normal_saliency_classes_listed = {}
                perturbed_saliency_classes_listed = {}
                distances_class_based = {}
                for key, values in normal_saliencies_for_multiple_classes.items():
                    normal_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_features_based(normal_saliencies_for_multiple_classes[key])
                    perturbed_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_features_based(perturbed_saliencies_for_multiple_classes[key])

                    normal_saliency_classes_listed[key], perturbed_saliency_classes_listed[key] = self.one_graph_all_together(normal_saliency_class0, perturbed_saliency_class0)
                    distances_class_based[key] = self.dist_explanation(normal_saliency_classes_listed[key], perturbed_saliency_classes_listed[key], top_k_features)
                    distances_class_based[key] = self.normalize_stability_distance(distances_class_based[key])
                    distances_class_based[key] = self.distance_thresholding_node_features(distances_class_based[key], importance_threshold)

                distance_list = []
                for key, value in distances_class_based.items():
                    distance_list.append(mean(value))

                return mean(distance_list)
            except:
                print("attributions are not in appropriate shape")

        elif style == "Node":
            try:
                normal_saliency_classes_listed = {}
                perturbed_saliency_classes_listed = {}
                distances_class_based = {}
                for key, values in normal_saliencies_for_multiple_classes.items():
                    normal_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_based(normal_saliencies_for_multiple_classes[key])
                    perturbed_saliencies_for_multiple_classes[key] = self.normalize_saliency_node_based(perturbed_saliencies_for_multiple_classes[key])

                    distances_class_based[key] = self.dist_explanation(normal_saliencies_for_multiple_classes[key], perturbed_saliencies_for_multiple_classes[key], top_k_features)
                    distances_class_based[key] = self.normalize_stability_distance(distances_class_based[key])
                    distances_class_based[key] = self.distance_thresholding_node_features(distances_class_based[key], importance_threshold)

                distance_list = []
                for key, value in distances_class_based.items():
                    distance_list.append(mean(value))

                return mean(distance_list)
            except:
                print("attributions are not in appropriate shape")

        elif style =="Edge":
            try:
                normal_saliency_classes_listed = {}
                perturbed_saliency_classes_listed = {}
                distances_class_based = {}
                for key, values in normal_saliencies_for_multiple_classes.items():
                    normal_saliencies_for_multiple_classes[key] = self.normalize_saliency_edge_based(normal_saliencies_for_multiple_classes[key])
                    perturbed_saliencies_for_multiple_classes[key] = self.normalize_saliency_edge_based(perturbed_saliencies_for_multiple_classes[key])

                    distances_class_based[key] = self.dist_explanation(normal_saliencies_for_multiple_classes[key], perturbed_saliencies_for_multiple_classes[key], top_k_features)
                    distances_class_based[key] = self.normalize_stability_distance(distances_class_based[key])
                    distances_class_based[key] = self.distance_thresholding_node_features(distances_class_based[key], importance_threshold)

                distance_list = []
                for key, value in distances_class_based.items():
                    distance_list.append(mean(value))

                return mean(distance_list)
            except:
                print("attributions are not in appropriate shape")
        else:
            print("Only node_feat, node, and edge are covered.")

In [ ]:
#stability = evalaution_of_xmethods_stability(GNN_Model, test_dataset)

In [ ]:
#perturbed_dataset = stability.perturb_node_features_of_dataset(test_dataset, 0.1, 0.1)

In [ ]:
#stability.my_stability(normal_saliency=importance_levels_for_feature_of_nodes_one_node_based,
#                       perturbed_saliency=importance_levels_for_feature_of_nodes_one_node_based_perturbed,
#                       top_k_features=10, importance_threshold=0.0000000001, style="Node")